## Aprendizaje de reglas de asociación

### Alumna: María Lucía Pappaterra

#### Tarea

* Obtener reglas de asociación entre películas en el dataset movielens (como si fuera recomendación!)

* Aplicar diferentes métricas de ordenamiento

* Hacer un pequeño informe

https://rpubs.com/vitidN/203264

Primero exporto todos los paquetes necesarios:

In [1]:
import numpy as np
import pandas as pd
import sys
import csv

from itertools import groupby
from efficient_apriori import apriori

### Explorando el data set

El data set tiene seis archivos .csv, veamos qué contiene cada uno.

In [2]:
links = pd.read_csv('ml-20m/links.csv')
movies = pd.read_csv('ml-20m/movies.csv')
ratings = pd.read_csv('ml-20m/ratings.csv')
tags = pd.read_csv('ml-20m/tags.csv')
genomescores = pd.read_csv('ml-20m/genome-scores.csv')
genometags = pd.read_csv('ml-20m/genome-tags.csv')

In [3]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [4]:
print('movies -- dimensions: {0};   size: {1}'.format(movies.shape, size(movies)))
movies.head()

movies -- dimensions: (27278, 3);   size: 4.50 MB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies.genres.unique()

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance', ...,
       'Action|Adventure|Animation|Fantasy|Horror',
       'Animation|Children|Comedy|Fantasy|Sci-Fi',
       'Animation|Children|Comedy|Western'], dtype=object)

In [6]:
print('links -- dimensions: {0};   size: {1}'.format(links.shape, size(links)))
links.head()

links -- dimensions: (27278, 3);   size: 0.65 MB


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
print('genome-scores -- dimensions: {0};   size: {1}'.format(genomescores.shape, size(genomescores)))
genomescores.head()

genome-scores -- dimensions: (11709768, 3);   size: 281.03 MB


,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [8]:
print('genome-tags -- dimensions: {0};   size: {1}'.format(genometags.shape, size(genometags)))
genometags.head()

genome-tags -- dimensions: (1128, 2);   size: 0.08 MB


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [9]:
print('ratings -- dimensions: {0};   size: {1}'.format(ratings.shape, size(ratings)))
ratings.head()

ratings -- dimensions: (20000263, 4);   size: 640.01 MB


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [10]:
ratings.rating.unique()

array([3.5, 4. , 3. , 4.5, 5. , 2. , 1. , 2.5, 0.5, 1.5])

In [11]:
print('tags -- dimensions: {0};   size: {1}'.format(tags.shape, size(tags)))
tags.head()

tags -- dimensions: (465564, 4);   size: 42.91 MB


,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [12]:
tags.tag.unique()

array(['Mark Waters', 'dark hero', 'noir thriller', ..., 'circle k',
       'This movie should have been called "How Cocaine Ruined Disney"',
       'topless scene'], dtype=object)

Los datos que nos interesan son los que se encuentran en **ratings**. 

In [13]:
ratings.rating.unique()

array([3.5, 4. , 3. , 4.5, 5. , 2. , 1. , 2.5, 0.5, 1.5])

Vemos que hay 10 valores de ranting posibles. Me interesan solo las películas con un rating mayor a un cierto umbral, ya que solo quiero recomendarle al usuario películas que le agraden. Por eso eliminamos del dataframe todas las películas que tengan un rating menor al rating mínimo fijado de antemano.

In [14]:
min_rat = 4.5 #rating mínimo

ratings = ratings[ratings.rating >= min_rat]
print('ratings -- dimensions: {0};   size: {1}'.format(ratings.shape, size(ratings)))
ratings.head()

ratings -- dimensions: (4433484, 4);   size: 177.34 MB


,userId,movieId,rating,timestamp
30,1,1196,4.5,1112484742
31,1,1198,4.5,1112484624
131,1,4993,5.0,1112484682
142,1,5952,5.0,1112484619
158,1,7153,5.0,1112484633


In [15]:
ratings.rating.unique()

array([4.5, 5. ])

Necesitamos decodificar el nombre de las películas, para eso usamos una columna de **movies**.

In [16]:
#decodificar el nombre de las películas:
ratings_df = pd.merge(ratings[['userId','movieId']], movies[['movieId','title']] ,on='movieId', how= "inner")

ratings_df.head()

,userId,movieId,title
0,1,1196,Star Wars: Episode V - The Empire Strikes Back...
1,2,1196,Star Wars: Episode V - The Empire Strikes Back...
2,3,1196,Star Wars: Episode V - The Empire Strikes Back...
3,5,1196,Star Wars: Episode V - The Empire Strikes Back...
4,7,1196,Star Wars: Episode V - The Empire Strikes Back...


In [17]:
ratings_df = ratings_df.sort_values( by='userId', axis=0, ascending=True, 
                                    inplace=False, kind='quicksort', 
                                    na_position='last')
print(ratings_df)

         userId  movieId                                              title
0             1     1196  Star Wars: Episode V - The Empire Strikes Back...
43502         1     4993  Lord of the Rings: The Fellowship of the Ring,...
63197         1     5952      Lord of the Rings: The Two Towers, The (2002)
80358         1     7153  Lord of the Rings: The Return of the King, The...
97248         1     8507                                      Freaks (1932)
...         ...      ...                                                ...
2229294  138493     2997                        Being John Malkovich (1999)
2058617  138493     5679                                   Ring, The (2002)
1258112  138493     1225                                     Amadeus (1984)
1453252  138493     5971       My Neighbor Totoro (Tonari no Totoro) (1988)
489328   138493     1206                         Clockwork Orange, A (1971)

[4433484 rows x 3 columns]


In [18]:
ratings = ratings_df.values[:,[0,2]]
print(ratings)

[[1 'Star Wars: Episode V - The Empire Strikes Back (1980)']
 [1 'Lord of the Rings: The Fellowship of the Ring, The (2001)']
 [1 'Lord of the Rings: The Two Towers, The (2002)']
 ...
 [138493 'Amadeus (1984)']
 [138493 'My Neighbor Totoro (Tonari no Totoro) (1988)']
 [138493 'Clockwork Orange, A (1971)']]


Cada **transacción** será el conjunto de todas las películas que un mismo usuario consideró con un rating mayor al que fijamos como mínimo y será almacenada en una lista.

### Métricas

Antes de derivar las reglas deseadas y para el análisis que sigue, necesitamos dejar en claro cuáles son las métricas asociadas a ellas.

El **soporte** de la regla $\{X\} \Rightarrow \{Y\}$ se define cómo $$ Sop = P(X \cup Y) = \dfrac{frec(X,Y)}{N} $$

donde $N$ es la cantidad de transacciones en la base de datos y $frec(X,Y)$ es la cantidad de transacciones en el conjunto de datos que contiene a $X$ y a $Y$ al mismo tiempo. Es decir, el soporte es la proporción de transacciones en la base de datos que contiene dicho conjunto de items.

Más soporte indica que la regla se encuentra en más transacciones, mientras que una regla con bajo soporte puede haber aparecido por casualidad.

La **confianza** de esta regla se define como: $$ Conf = P(Y|X) = \dfrac{frec(X,Y)}{frec(X)}$$

esto es, el porcentaje de transacciones que contienen $Y$, entre las transacciones que contienen $X$.

Más confianza indica mayor probabilidad de que la regla sea cierta para una transacción. Si una regla tiene baja confianza, es probable que no exista relación entre antecedente y consecuente.

Tanto el soporte como la confianza son probabilidades, por lo tanto sus valores varían entre 0 y 1. 

El **lift** indica la proporción entre el soporte observado de un conjunto de ítems respecto del soporte teórico de ese conjunto dado el supuesto de independencia. Y está dado por las siguientes fórmulas equivalentes: $$ Lift =   \dfrac{Sop(X \Rightarrow Y)}{Sop(X) \times Sop(Y)} = \dfrac{P(X \cup Y)}{P(X) \times P(Y)} = \dfrac{N \times frec(X,Y)}{frec(X) \times frec(Y)}$$

Más lift indica menor probabilidad de que la regla sea una casualidad.

La **convicción** expresa que tan independientes son las variables $X$ y $¬Y$, 

$$ Conv = \dfrac{1 - Sop(Y)}{1 - Conf(X \Rightarrow Y)} = \dfrac{frec(X) \times frec(¬Y)}{frec(X, ¬Y)} $$

donde $frec(¬Y)$ es la cantidad de transacciones en la base de datos que no contienen a $Y$ y $frec(X, ¬Y)$ es la cantidad de transacciones en la base de datos que contienen a $X$ y no contienen a $Y$.

La convicción va de 1 a infinito (si la confianza es 1, la convicción es infinita, no 0). Más convicción indica mayor grado de implicación. Altos valores para convicción (cuando $frec(X, ¬Y)$ tiende a cero) afirman la convicción de que esta regla representa una causalidad.

### Derivación de las reglas

Las mismas serán generadas con el **algoritmo apriori**, https://pypi.org/project/efficient-apriori/

Como tenemos muchos datos, para almacenar las transacciones debemos usar un generador de listas en lugar de una lista de listas. El generador que defino a continuación va creando cada transacción de a una para no usar toda la memoria.

In [20]:
def transactions_generator():
    for userId, movie_object in groupby(ratings, lambda x: x[0]):
        yield [item[1] for item in movie_object]

A continuación corremos el algoritmo apriori e imprimimos las reglas generadas con sus métricas asociadas. 

Elegí una confianza mínima de $0.55$, pues quería tener una confianza mayor al $50\%$, pero también generar reglas interesantes y no tan obvias. 

Para el soporte mínimo probé varios valores, siendo $0.06$ el mínimo posible para el cuál el tiempo de ejecución es razonable.

In [21]:
trans_gen = transactions_generator()

itemsets, rules = apriori(trans_gen, min_support=0.06,  min_confidence=0.55)

rules=sorted(rules, key=lambda rule: rule.confidence)
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...

{Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.555, supp: 0.092, lift: 4.187, conv: 1.950)
{Usual Suspects, The (1995)} -> {Shawshank Redemption, The (1994)} (conf: 0.556, supp: 0.109, lift: 1.872, conv: 1.584)
{Seven (a.k.a. Se7en) (1995)} -> {Shawshank Redemption, The (1994)} (conf: 0.557, supp: 0.073, lift: 1.873, conv: 1.586)
{Star Wars: Episode IV - A New Hope (1977), Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.563, supp: 0.068, lift: 3.661, conv: 1.936)
{Shawshank Redemption, The (1994), Silence of the Lambs, The (1991)} -> {Pulp Fiction (1994)} (conf: 0.569, supp: 0.062, lift: 2.179, conv: 1.714)
{Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.579, supp: 0.089, lift: 2.858, conv: 1.893)
{Shawshank Redemption, The (199

### Análisis  y Conclusiones

Todas las reglas derivadas tienen un **soporte** muy bajo. Casi todas tienen el primer dígito decimal igual a $0$, y las de soporte más alto primer dígito decimal igual a $1$, ninguna supera el $0.13$. Esto indica que las reglas derivadas se encuentran en pocas transacciones de la base de datos, ninguna de ellas está presente en el $13\%$ o más de las transacciones.

Las reglas obtenidas están ordenadas según su **confianza** de menor a mayor, es decir que las primeras reglas son menos confiables que las últimas. La menor confianza obtenida fue de $0.555$, mientras que la mayor de $0.896$.

Los valores de **lift** son mayores a 1 en todos los casos. Esto significa que, para cada regla $\{X \} \Rightarrow \{Y \}$ derivada, $X$ e $Y$ aparecen juntos una cantidad de veces superior a lo esperado bajo condiciones de independencia, indicando que la aparición de $X$ tiene un efecto positivo sobre la aparición de $Y$.

Los valores de **convicción** también son altos en todos los casos, indicando causalidad entre $X$ e $Y$.

Muchas de las reglas generadas tienen sentido intuitivamente. Como las que relacionan películas que pertenecen a una misma saga: las del Señor de los Anillos, las de Star Wars, las del Padrino, las de Indiana Jones, etc... O las que relacionan películas de un mismo director (por ejemplo: {Reservoir Dogs (1992)} -> {Pulp Fiction (1994)}). Esto me hace pensar que el algoritmo está funcionando correctamente.

Algunas reglas que me resultaron interesantes y no tan obvias son:

* {Pulp Fiction (1994), Usual Suspects, The (1995)} -> {Shawshank Redemption, The (1994)}
* {Pulp Fiction (1994), Silence of the Lambs, The (1991)} -> {Shawshank Redemption, The (1994)}
* {Seven (a.k.a. Se7en) (1995)} -> {Pulp Fiction (1994)}
* {Shawshank Redemption, The (1994), Usual Suspects, The (1995)} -> {Pulp Fiction (1994)}
* {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} -> {Star Wars: Episode IV - A New Hope (1977)}
* {Shawshank Redemption, The (1994), Silence of the Lambs, The (1991)} -> {Pulp Fiction (1994)}
* {Star Wars: Episode IV - A New Hope (1977), Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)}
* {Seven (a.k.a. Se7en) (1995)} -> {Shawshank Redemption, The (1994)}
* {Usual Suspects, The (1995)} -> {Shawshank Redemption, The (1994)}

En general diría que todas las reglas encontradas están bien validadas por las métricas, pero para poder profundizar más el análisis serían necesarias diversas tareas de post-procesamiento para eliminar reglas no interesantes, podar reglas redundantes y conservar las reglas con mayor grado de generalidad.